In [1]:
P = Primes()
ZR.<x> = PolynomialRing(ZZ)
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
C.<x> = PolynomialRing(CC)
S = PowerSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m,x):
    f = J(n+2,m,x)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m,x): 
    f=J(n,m,x)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m,x):
    return xjNewStep(n-1,m,x)
def xjNewCoefficient(n,m,x):
    if n>0:return (xjNew(n,m,x)-xjNew(n-1,m,x))/x^n
    if n==0:return 1
def xjNewTest(n,m,x):
    return D(xjNewStep(n-1,m,x))
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

def prime_n(a):return P.unrank(a-1)

def primeFactors(n):
    answer=[]
    lf=list(n.factor())
    length=len(lf)
    for k in [0..length-1]:
        pair=lf[k]
        (base,exponent)=pair
        answer=answer+[[base]]
    return(flatten(answer))

def primeFactorsInList(lst):
    ld=len(lst)
    primelist=[]
    for k in [0..ld-1]:
        a=ZZ(lst[k])
        primelist=primelist+[primeFactors(a)]
    primelist=flatten(primelist)
    primelist=list(set(primelist))
    return primelist

def den(a):
    if a in QQ:
        if a in ZZ:return 1
        return a.denominator()
def listdens(lst):
    return list(den(a) for a in lst)
def polynomialdenominator(poly):return lcm(listdens(poly))
def polynomialnumerator(poly):
    return expand(poly*polynomialdenominator(poly))
def num(a):return a.numerator()
def prime(n):
    P=Primes()
    return P.unrank(n-1)
def inclusiveListPrimesTo(n):
    if is_prime(n):
        return list(primes(n+1))
    else:
        return list(primes(n))
def chunk(k,list):
    chnk=[]
    for n in [k..len(list)-1]:
        chnk=chnk+[list[n]]
    return chnk
def drop(m,lst): return list(Set(lst).difference({m}))
def sort(list):
    answr=[]
    chnk=list
    for n in [0..len(list)-1]:
        m = min(chnk)
        answr=answr+[m]
        chnk=drop(m,chnk)
    return answr
def ord(p,n):return QQ.valuation(p)(n)
def base_conversion(number, base_from, base_to):  # by Vishnu Namboothiri
# https://ask.sagemath.org/question/9657/how-to-output-decimal-numbers-in-different-radix/
    if(base_from==10):
        return stripQuotationMarks((number.str(base=base_to)))
    else:
        decimal = Integer(str(number), base=base_from)
        return (stripQuotationMarks(decimal.str(base=base_to)))
def lastFactorWithIntegralFactors(polynomial):
    return polynomialnumerator(lastfactor(polynomial))
def numericalFactorWithIntegralFactors(polynomial):
    nf=numericalfactor(polynomial);
    pd=polynomialdenominator(lastfactor(polynomial))
    return nf/pd


def primeDivisors(n):
    pd=[]
    n=n
    fn=factor(n)
    ln=len(fn)
    for k in [0..ln-1]:
        part = fn[k]
        (prime,exponent) = part
        pd = pd + [prime]
    return pd 
def fOverField(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return ppoly

###################### following two routines disagree ####################
def factorOverField(f,prime,power):
    answer=[]
    T = GF(prime^power,'t')
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return fp

def factorOverField2(poly,prime,power,j):
    sd = leastSplittingDegree(poly,prime,bound)
    T=GF(prime^j)
    G.<t>= GF(prime^sd)
    U.<x> = T[]
    ppoly=U(poly)
    fpp=ppoly.factor()
    return fpp

############################

#######################################################
# splitting tests below fail at  e.g. f = x + 1 
############################################################

def splitsTF(f,prime,power):
    T = GF(prime^power)
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx > 1:return(False)
    return(True)

def leastSplittingDegree(f,prime,bound):
    for power in [1..bound]:
        tf=splitsTF(f,prime,power)
        if tf == True:return power
    return -1

#########################################################

def firstNprimes(n):
    P=Primes()
    first=[]
    for k in [0..n-1]:
        first=first+[P.unrank(k)]
    return first

def slopeExistsTF(pairlist):
    lenth=len(pairlist)
    if lenth<3:return False
    slopes=[]
    for k in [1..lenth-1]:
        pair1=pairlist[k-1]
        abscissa1=pair1[0]
        ordinate1=pair1[1]
        pair2=pairlist[k]
        abscissa2=pair2[0]
        ordinate2=pair2[1]
        dx=abscissa2-abscissa1
        if dx == 0:return False
        dy=ordinate2-ordinate1
        slope=dy/dx
        slopes=slopes+[slope]
    slopes=Set(slopes)
    if len(slopes) == 1:
        return [True,slopes[0]]
    if len(slopes) > 1:
        return False 
    
def zeroOfLinearMonomial(monomial,x):return x-monomial

def totalDegreeOverField(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    td=0
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        td=td+exponent
    return td

def logsAndMultiplicitiesOverField(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:data=data+[[i,exponent]]
    return data

def generatingZeroLogsOverField(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:
                        if mod(i,prime)>0:
                            data=data+[i]
    data=data.sort()
    return data



def generatingZeroLogsAndMultiplicitiesOverField(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:
                        if mod(i,prime)>0:
                            data=data+[[i,exponent]]
    return data




def logsAndMultiplicitiesOverFieldTest(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    print gf
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:data=data+[[i,exponent]]
    return data

def logsAndMultiplicitiesOverFieldTest(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    print gf
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:data=data+[[i,exponent]]
    return data



def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data

def padListOnRightToLengthK(listo,k):
    length=len(listo)
    padded=listo
   
    if length<k:
        for j in [length..k-1]:
            padded=padded+[0]
           
        return padded
    return listo

def testCandidateZeroLogOverField(f,prime,power,candidate):
    tf=False
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    a=t^candidate
    expr3=sage_eval(expr,locals={'x':a})
    if expr3 == 0:tf=True
    return tf

def digitsToInteger(digitlist,prime):
    lenth=len(digitlist)
    sum=0
    for k in [0..lenth-1]:
        sum=sum+prime^k*digitlist[k]
    return sum

def minimumIntegerRepresentedInList(lstlst,prime):
    answer=Infinity
    ln=len(lstlst)
    for k in [0..ln-1]:
        lst=lstlst[k]
        n=digitsToInteger(lst,prime)
        if n < answer:answer=n
    return answer

def lastElement(lst):
    ln=len(lst)
    return lst[ln-1]

def push(lst):
    if len(lst)<2:return lst
    answer=[lastElement(lst)]
    for k in [0..len(lst)-2]:
        answer=flatten([answer]+[lst[k]])
    return answer

def cyclicPermutationEquivalenceClass(lst):
    if lst==[]:return lst
    answer=[lst]
    lnth=len(lst)
    for k in [0..lnth-2]:
        ple=push(lastElement(answer))
        answer=answer+[ple]
    return answer

def bestRepresentativeOfCyclicEquivalenceClass(lst,prime):
    cpec=cyclicPermutationEquivalenceClass(lst)
    minm=minimumIntegerRepresentedInList(cpec,prime)
    lcpec=len(lst) # ... = len(cpec)
    for k in [0..lcpec-1]:
        ck=cpec[k]
        dti=digitsToInteger(ck,prime)
        if minm==dti:return ck
    return "error"

def directZeroLogOverField(f,prime,power):
    tf=False
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    answer=[]
    expr3=sage_eval(expr,locals={'x':0})
    if expr3 == 0:
        answer=[0]
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            answer=answer+[i]
    return answer

def reduce(f,prime):
    poly=polynomialnumerator(D(f))
    
    ls=list(poly)
    
    def modulo(n):
        return ZZ(mod(n,prime))
    def listmod(list):
        return map(modulo,list)
    lmls = listmod(ls)
    sm=0
    for j in [0..len(lmls)-1]:
        sm=sm+lmls[j]*x^j
    return sm


mt=[ #McKay-Thompson series w. a(0) = 24.
[-1, 1],
[0, 24],
[1, 276],
[2, 2048],
[3, 11202],
[4, 49152],
[5, 184024],
[6, 614400],
[7, 1881471],
[8, 5373952],
[9, 14478180],
[10, 37122048],
[11, 91231550],
[12, 216072192],
[13, 495248952],
[14, 1102430208],
[15, 2390434947],
[16, 5061476352],
[17, 10487167336],
[18, 21301241856],
[19, 42481784514],
[20, 83300614144],
[21, 160791890304],
[22, 305854488576],
[23, 573872089212],
[24, 1063005978624],
[25,1945403602764],
[26, 3519965179904],
[27, 6300794030460],
[28, 11164248047616],
[29, 19591528119288],
[30, 34065932304384],
[31, 58718797964805],
[32, 100372723007488],
[33, 170215559855424],
[34, 286470013685760],
[35, 478625723149576],
[36, 794110053826560],
[37, 1308745319975256],
[38, 2143055278039040],
[39, 3487563372381816],
[40, 5641848336678912],
[41, 9074553043554568],
[42, 14515166263443456],
[43, 23093778743102262],
[44, 36552977852071936],
[45, 57567784186189368],
[46, 90226777113575424],
[47, 140752796480416011],
[48, 218578429975461888],
[49, 337945040343588276],
[50, 520271697765971968],
[51, 797652526220573580],
[52, 1218002527825723392],
[53, 1852604006634050072],
[54, 2807138079496716288],
[55, 4237760460302936433],
[56, 6374456847628238848],
[57, 9554873766107770560],
[58, 14273181657059143680],
[59, 21250450411204068910],
[60, 31535729115847852032],
[61, 46650835290143061624],
[62, 68797209365301886976],
[63, 101150679669913197462],
[64, 148280443106626633728],
[65, 216743142763626253712],
[66, 315923191441199824896],
[67, 459218611940943755226],
[68, 665710603285072019456],
[69, 962508846974918603904],
[70, 1388038765923851599872],
[71, 1996639069403279491427],
[72, 2864978197116521938944],
[73, 4100990608911708903432],
[74, 5856297079648098807808],
[75, 8343432715970391209502],
[76, 11859696700297921757184],
[77, 16820105145987654631552],
[78, 23802835313046730063872],
[79, 33611779636250175278886],
[80, 47362494062244172660736],
[81, 66600077798590855556532],
[82, 93460562353103053049856],
[83, 130891485964083426534122],
[84, 182952844329494181838848],
[85, 255227018229957765044016],
[86, 355376219286719031156736],
[87, 493899311443420857952845],
[88, 685157678128482627354624],
[89, 948763597225844233250504],
[90, 1311456320500974276980736],
[91, 1809633323386495729057992],
[92, 2492760414984152205361152],
[93, 3427959082742197097793024],
[94, 4706168520874397834575872],
[95, 6450411048962389429976770],
[96, 8826863296640622526464000],
[97, 12059665023346371597383976],
[98, 16450700321708764061517824],
[99, 22405985191605778318221966],
[100, 30470821082605141257437184]]

def sumOfRoots(f,prime,power):
    answer=0
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    #U.<x> = T[] -- this line gave an error
    poly = U(f)
    expr=str(poly)
    var('x')
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            answer=answer+a
    return answer

def productOfNonzeroRoots(f,prime,power):
    answer=1
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    #U.<x> = T[] -- this line gave an error
    poly = U(f)
    expr=str(poly)
    var('x')
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            answer=answer*a
    return answer

def roots(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    #U.<x> = T[] -- this line gave an error
    poly = U(f)
    expr=str(poly)
    var('x')
    expr3=sage_eval(expr,locals={'x':0})
    if expr3==0:answer=answer+[0]
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            answer=answer+[a]
    return answer

def numericalTermOverFieldNoPrint(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    xproduct=1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        xproduct=xproduct*monomial^exponent
    return g/xproduct

def polynomialDegree(poly):
    return len(poly.list())-1

def leadingCoefficient(poly):
    dp=D(poly)
    deg=polynomialDegree(dp)
    return dp.list()[deg]

def testRootLogTimesPrimeCn(f,prime,power):
    tf=True
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    fp=poly.factor()
    lnf = len(fp) - 1
    flag = 1
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            j=i*prime
            b=t^j
            expr4=sage_eval(expr,locals={'x':b})
            if expr4!=0:
                tf=False
                print (i,j)
                
    return tf

def basicRoots(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    fp=poly.factor()
    lnf = len(fp) - 1
    flag = 1
    for i in [1..prime^power-1]:
        if mod(i,prime)!=0:
            a=t^i
            expr3=sage_eval(expr,locals={'x':a})
            if expr3 == 0:answer=answer+[i]
    return answer

def basicRootsDigits(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    fp=poly.factor()
    lnf = len(fp) - 1
    flag = 1
    for i in [1..prime^power-1]:
        if mod(i,prime)!=0:
            a=t^i
            expr3=sage_eval(expr,locals={'x':a})
            if expr3 == 0:answer=answer+[padListOnRightToLengthK(i.digits(prime),power)]
    return answer

def basicLogsAndMultiplicitiesOverField(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if mod(i,prime)!=0:
                        if b^i==xm:data=data+[[i,exponent]]
    return data.sort()

def totalMultiplicityOverField(f,prime,power):
    tm=0
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:tm=tm+exponent
    return tm

def testRootLogTimesPrimeCn(f,prime,power):
    tf=True
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    fp=poly.factor()
    lnf = len(fp) - 1
    flag = 1
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            j=i*prime
            b=t^j
            expr4=sage_eval(expr,locals={'x':b})
            if expr4!=0:
                tf=False
                print (i,j)
                
    return tf

In [ ]:
## printed output has been snipped

import pickle
wfile=open('/Users/barrybrent/run1jun21no1.txt','a')
data=[]
for m in [3..500]:
    print m
    data=data+[[m,normalJ(200,m)]]
s=pickle.dumps(str(data))
wfile.write(s)
wfile.close()
#########################
import pickle
rfile = open('/Users/barrybrent/run1jun21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

polydata=[]

wfile = ## printed output has been snipped

import pickle
wfile=open('/Users/barrybrent/run1jun21no1.txt','a')
data=[]
for m in [3..500]:
    print m
    data=data+[[m,normalJ(200,m)]]
s=pickle.dumps(str(data))
wfile.write(s)
wfile.close()
#########################
import pickle
rfile = open('/Users/barrybrent/run1jun21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

polydata=[]

wfile = open('/Users/barrybrent/run14jun21no14.txt','a') # xJ polynomials
for n in [0..200]:
    data=[]
    for k in [0..2*n+1]:
        print (n,k)
        m = s[k][0]
        poly=x*s[k][1] 
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf*m^(2*n)]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n-1,rl]] 

    
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close() # print-output snippeds
for n in [0..200]:
    data=[]
    for k in [0..2*n+1]:
        print (n,k)
        m = s[k][0]
        poly=x*s[k][1] 
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf*m^(2*n)]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n-1,rl]] 

    
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close() # print-output snipped

sum

In [2]:
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') #Jfn
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
len(s)

201

In [2]:
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=7
pairs=[]
for k in [1..200]:
    n=s[k][0]
    if (1==1)&(n>prime):
        poly=D(s[k][1])
        polynum=ZR(polynomialnumerator(D(poly)))
        sd=leastSplittingDegree(polynum,prime,bound)
        pair=(mod(n,prime),sd)
        pairs=pairs+[pair]
print sort(union(pairs))
        

[(0, 4), (1, 4), (2, 2), (3, 2), (4, 6), (5, 5), (6, 1)]


In [3]:
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=7
residue=0
sdvalue=4
badn=[]
count=0
flag=1
template=[]
for k in [1..200]:
    n=s[k][0]
    if (n>prime-1)&(mod(n,prime)==residue):
        poly=D(s[k][1])
        polynum=ZR(polynomialnumerator(D(poly)))
        sd=leastSplittingDegree(polynum,prime,bound)
    
        if (sd==sdvalue)&(flag==1):
            template=basicRoots(polynum,prime,sd)
            flag=0
    
        if (sd==sdvalue):
            count=1+count
            br=basicRoots(polynum,prime,sd)
            if br!=template:badn=badn+[n]
            lm=logsAndMultiplicitiesOverField(polynum,prime,sd)
            tm=totalMultiplicityOverField(polynum,prime,sd)
            print "------------------------------------------------------------------------"
            print "n: ",n," n.digits(prime): ",n.digits(prime)," sd: ",sd," prime: ",prime," n mod prime: ",mod(n,prime)
            print "basic roots:";print br
            print "root-logs and multiplicities:";print sorted(lm)
            print "total multiplicity of roots: ",tm
print "&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&"
if count==0:print "null"
print " bad n: ",badn;print "template: ",template

------------------------------------------------------------------------
n:  7  n.digits(prime):  [0, 1]  sd:  4  prime:  7  n mod prime:  0
basic roots:
[325, 400, 600, 800, 1075, 1525, 1600, 1800, 2000]
root-logs and multiplicities:
[[325, 1], [400, 3], [600, 1], [800, 2], [1075, 1], [1525, 1], [1600, 3], [1800, 1], [2000, 2], [2275, 1]]
total multiplicity of roots:  16
------------------------------------------------------------------------
n:  14  n.digits(prime):  [0, 2]  sd:  4  prime:  7  n mod prime:  0
basic roots:
[325, 400, 600, 800, 1075, 1200, 1525, 1600, 1800, 2000, 2400]
root-logs and multiplicities:
[[325, 1], [400, 5], [600, 1], [800, 4], [1075, 1], [1200, 2], [1525, 1], [1600, 5], [1800, 1], [2000, 4], [2275, 1], [2400, 2]]
total multiplicity of roots:  28
------------------------------------------------------------------------
n:  21  n.digits(prime):  [0, 3]  sd:  4  prime:  7  n mod prime:  0
basic roots:
[325, 400, 600, 800, 1075, 1200, 1525, 1600, 1800, 2000, 240

------------------------------------------------------------------------
n:  147  n.digits(prime):  [0, 0, 3]  sd:  4  prime:  7  n mod prime:  0
basic roots:
[325, 400, 600, 800, 1075, 1200, 1525, 1600, 1800, 2000, 2400]
root-logs and multiplicities:
[[325, 1], [400, 43], [600, 1], [800, 42], [1075, 1], [1200, 40], [1525, 1], [1600, 43], [1800, 1], [2000, 42], [2275, 1], [2400, 40]]
total multiplicity of roots:  256
------------------------------------------------------------------------
n:  154  n.digits(prime):  [0, 1, 3]  sd:  4  prime:  7  n mod prime:  0
basic roots:
[325, 400, 600, 800, 1075, 1200, 1525, 1600, 1800, 2000, 2400]
root-logs and multiplicities:
[[325, 1], [400, 45], [600, 1], [800, 44], [1075, 1], [1200, 42], [1525, 1], [1600, 45], [1800, 1], [2000, 44], [2275, 1], [2400, 42]]
total multiplicity of roots:  268
------------------------------------------------------------------------
n:  161  n.digits(prime):  [0, 2, 3]  sd:  4  prime:  7  n mod prime:  0
basic roots:

In [4]:
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=7
residue=0
sdvalue=4
badn=[]
count=0
flag=1
template=[]
for k in [1..200]:
    n=s[k][0]
    if (n>prime)&(mod(n,prime)==residue):
        poly=D(s[k][1])
        polynum=ZR(polynomialnumerator(D(poly)))
        sd=leastSplittingDegree(polynum,prime,bound)
    
        if (sd==sdvalue)&(flag==1):
            template=basicRoots(polynum,prime,sd)
            flag=0
    
        if (sd==sdvalue):
            count=1+count
            br=basicRoots(polynum,prime,sd)
            if br!=template:badn=badn+[n]
            lm=logsAndMultiplicitiesOverField(polynum,prime,sd)
            tm=totalMultiplicityOverField(polynum,prime,sd)
            print "------------------------------------------------------------------------"
            print "n: ",n," n.digits(prime): ",n.digits(prime)," sd: ",sd," prime: ",prime," n mod prime: ",mod(n,prime)
            print "basic roots:";print br
            print "root-logs and multiplicities:";print sorted(lm)
            print "total multiplicity of roots: ",tm
print "&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&"
if count==0:print "null"
print " bad n: ",badn;print "template: ",template

------------------------------------------------------------------------
n:  14  n.digits(prime):  [0, 2]  sd:  4  prime:  7  n mod prime:  0
basic roots:
[325, 400, 600, 800, 1075, 1200, 1525, 1600, 1800, 2000, 2400]
root-logs and multiplicities:
[[325, 1], [400, 5], [600, 1], [800, 4], [1075, 1], [1200, 2], [1525, 1], [1600, 5], [1800, 1], [2000, 4], [2275, 1], [2400, 2]]
total multiplicity of roots:  28
------------------------------------------------------------------------
n:  21  n.digits(prime):  [0, 3]  sd:  4  prime:  7  n mod prime:  0
basic roots:
[325, 400, 600, 800, 1075, 1200, 1525, 1600, 1800, 2000, 2400]
root-logs and multiplicities:
[[325, 1], [400, 7], [600, 1], [800, 6], [1075, 1], [1200, 4], [1525, 1], [1600, 7], [1800, 1], [2000, 6], [2275, 1], [2400, 4]]
total multiplicity of roots:  40
------------------------------------------------------------------------
n:  28  n.digits(prime):  [0, 4]  sd:  4  prime:  7  n mod prime:  0
basic roots:
[325, 400, 600, 800, 1075

------------------------------------------------------------------------
n:  154  n.digits(prime):  [0, 1, 3]  sd:  4  prime:  7  n mod prime:  0
basic roots:
[325, 400, 600, 800, 1075, 1200, 1525, 1600, 1800, 2000, 2400]
root-logs and multiplicities:
[[325, 1], [400, 45], [600, 1], [800, 44], [1075, 1], [1200, 42], [1525, 1], [1600, 45], [1800, 1], [2000, 44], [2275, 1], [2400, 42]]
total multiplicity of roots:  268
------------------------------------------------------------------------
n:  161  n.digits(prime):  [0, 2, 3]  sd:  4  prime:  7  n mod prime:  0
basic roots:
[325, 400, 600, 800, 1075, 1200, 1525, 1600, 1800, 2000, 2400]
root-logs and multiplicities:
[[325, 1], [400, 47], [600, 1], [800, 46], [1075, 1], [1200, 44], [1525, 1], [1600, 47], [1800, 1], [2000, 46], [2275, 1], [2400, 44]]
total multiplicity of roots:  280
------------------------------------------------------------------------
n:  168  n.digits(prime):  [0, 3, 3]  sd:  4  prime:  7  n mod prime:  0
basic roots:

In [5]:
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=7
residue=1
sdvalue=4
badn=[]
count=0
flag=1
template=[]
for k in [1..200]:
    n=s[k][0]
    if (n>prime)&(mod(n,prime)==residue):
        poly=D(s[k][1])
        polynum=ZR(polynomialnumerator(D(poly)))
        sd=leastSplittingDegree(polynum,prime,bound)
    
        if (sd==sdvalue)&(flag==1):
            template=basicRoots(polynum,prime,sd)
            flag=0
    
        if (sd==sdvalue):
            count=1+count
            br=basicRoots(polynum,prime,sd)
            if br!=template:badn=badn+[n]
            lm=logsAndMultiplicitiesOverField(polynum,prime,sd)
            tm=totalMultiplicityOverField(polynum,prime,sd)
            print "------------------------------------------------------------------------"
            print "n: ",n," n.digits(prime): ",n.digits(prime)," sd: ",sd," prime: ",prime," n mod prime: ",mod(n,prime)
            print "basic roots:";print br
            print "root-logs and multiplicities:";print sorted(lm)
            print "total multiplicity of roots: ",tm
print "&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&"
if count==0:print "null"
print " bad n: ",badn;print "template: ",template

------------------------------------------------------------------------
n:  8  n.digits(prime):  [1, 1]  sd:  4  prime:  7  n mod prime:  1
basic roots:
[75, 400, 800, 1200, 1275, 1600, 1725, 2000, 2400]
root-logs and multiplicities:
[[75, 1], [400, 2], [525, 1], [800, 2], [1200, 2], [1275, 1], [1600, 2], [1725, 1], [2000, 2], [2400, 2]]
total multiplicity of roots:  16
------------------------------------------------------------------------
n:  15  n.digits(prime):  [1, 2]  sd:  4  prime:  7  n mod prime:  1
basic roots:
[75, 400, 800, 1200, 1275, 1600, 1725, 2000, 2400]
root-logs and multiplicities:
[[75, 1], [400, 4], [525, 1], [800, 4], [1200, 4], [1275, 1], [1600, 4], [1725, 1], [2000, 4], [2400, 4]]
total multiplicity of roots:  28
------------------------------------------------------------------------
n:  22  n.digits(prime):  [1, 3]  sd:  4  prime:  7  n mod prime:  1
basic roots:
[75, 400, 800, 1200, 1275, 1600, 1725, 2000, 2400]
root-logs and multiplicities:
[[75, 1], [400,

------------------------------------------------------------------------
n:  162  n.digits(prime):  [1, 2, 3]  sd:  4  prime:  7  n mod prime:  1
basic roots:
[75, 400, 800, 1200, 1275, 1600, 1725, 2000, 2400]
root-logs and multiplicities:
[[75, 1], [400, 46], [525, 1], [800, 46], [1200, 46], [1275, 1], [1600, 46], [1725, 1], [2000, 46], [2400, 46]]
total multiplicity of roots:  280
------------------------------------------------------------------------
n:  169  n.digits(prime):  [1, 3, 3]  sd:  4  prime:  7  n mod prime:  1
basic roots:
[75, 400, 800, 1200, 1275, 1600, 1725, 2000, 2400]
root-logs and multiplicities:
[[75, 1], [400, 48], [525, 1], [800, 48], [1200, 48], [1275, 1], [1600, 48], [1725, 1], [2000, 48], [2400, 48]]
total multiplicity of roots:  292
------------------------------------------------------------------------
n:  176  n.digits(prime):  [1, 4, 3]  sd:  4  prime:  7  n mod prime:  1
basic roots:
[75, 400, 800, 1200, 1275, 1600, 1725, 2000, 2400]
root-logs and mult

In [6]:
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=7
residue=2
sdvalue=2
badn=[]
count=0
flag=1
template=[]
for k in [1..200]:
    n=s[k][0]
    if (n>prime)&(mod(n,prime)==residue):
        poly=D(s[k][1])
        polynum=ZR(polynomialnumerator(D(poly)))
        sd=leastSplittingDegree(polynum,prime,bound)
    
        if (sd==sdvalue)&(flag==1):
            template=basicRoots(polynum,prime,sd)
            flag=0
    
        if (sd==sdvalue):
            count=1+count
            br=basicRoots(polynum,prime,sd)
            if br!=template:badn=badn+[n]
            lm=logsAndMultiplicitiesOverField(polynum,prime,sd)
            tm=totalMultiplicityOverField(polynum,prime,sd)
            print "------------------------------------------------------------------------"
            print "n: ",n," n.digits(prime): ",n.digits(prime)," sd: ",sd," prime: ",prime," n mod prime: ",mod(n,prime)
            print "basic roots:";print br
            print "root-logs and multiplicities:";print sorted(lm)
            print "total multiplicity of roots: ",tm
print "&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&"
if count==0:print "null"
print " bad n: ",badn;print "template: ",template

------------------------------------------------------------------------
n:  9  n.digits(prime):  [2, 1]  sd:  2  prime:  7  n mod prime:  2
basic roots:
[4, 8, 16, 24, 32, 40, 48]
root-logs and multiplicities:
[[4, 2], [8, 2], [16, 3], [24, 2], [28, 2], [32, 2], [40, 3], [48, 2]]
total multiplicity of roots:  18
------------------------------------------------------------------------
n:  16  n.digits(prime):  [2, 2]  sd:  2  prime:  7  n mod prime:  2
basic roots:
[4, 8, 16, 24, 32, 40, 48]
root-logs and multiplicities:
[[4, 2], [8, 4], [16, 5], [24, 4], [28, 2], [32, 4], [40, 5], [48, 4]]
total multiplicity of roots:  30
------------------------------------------------------------------------
n:  23  n.digits(prime):  [2, 3]  sd:  2  prime:  7  n mod prime:  2
basic roots:
[4, 8, 16, 24, 32, 40, 48]
root-logs and multiplicities:
[[4, 2], [8, 6], [16, 7], [24, 6], [28, 2], [32, 6], [40, 7], [48, 6]]
total multiplicity of roots:  42
-----------------------------------------------------

------------------------------------------------------------------------
n:  191  n.digits(prime):  [2, 6, 3]  sd:  2  prime:  7  n mod prime:  2
basic roots:
[4, 8, 16, 24, 32, 40, 48]
root-logs and multiplicities:
[[4, 2], [8, 54], [16, 55], [24, 54], [28, 2], [32, 54], [40, 55], [48, 54]]
total multiplicity of roots:  330
------------------------------------------------------------------------
n:  198  n.digits(prime):  [2, 0, 4]  sd:  2  prime:  7  n mod prime:  2
basic roots:
[4, 8, 16, 24, 32, 40, 48]
root-logs and multiplicities:
[[4, 2], [8, 56], [16, 57], [24, 56], [28, 2], [32, 56], [40, 57], [48, 56]]
total multiplicity of roots:  342
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
 bad n:  []
template:  [4, 8, 16, 24, 32, 40, 48]


In [7]:
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=7
residue=3
sdvalue=2
badn=[]
count=0
flag=1
template=[]
for k in [1..200]:
    n=s[k][0]
    if (n>prime)&(mod(n,prime)==residue):
        poly=D(s[k][1])
        polynum=ZR(polynomialnumerator(D(poly)))
        sd=leastSplittingDegree(polynum,prime,bound)
    
        if (sd==sdvalue)&(flag==1):
            template=basicRoots(polynum,prime,sd)
            flag=0
    
        if (sd==sdvalue):
            count=1+count
            br=basicRoots(polynum,prime,sd)
            if br!=template:badn=badn+[n]
            lm=logsAndMultiplicitiesOverField(polynum,prime,sd)
            tm=totalMultiplicityOverField(polynum,prime,sd)
            print "------------------------------------------------------------------------"
            print "n: ",n," n.digits(prime): ",n.digits(prime)," sd: ",sd," prime: ",prime," n mod prime: ",mod(n,prime)
            print "basic roots:";print br
            print "root-logs and multiplicities:";print sorted(lm)
            print "total multiplicity of roots: ",tm
print "&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&"
if count==0:print "null"
print " bad n: ",badn;print "template: ",template

------------------------------------------------------------------------
n:  10  n.digits(prime):  [3, 1]  sd:  2  prime:  7  n mod prime:  3
basic roots:
[1, 8, 12, 16, 24, 25, 31, 32, 36, 40, 48]
root-logs and multiplicities:
[[1, 1], [7, 1], [8, 2], [12, 1], [16, 3], [24, 2], [25, 1], [31, 1], [32, 2], [36, 1], [40, 3], [48, 2]]
total multiplicity of roots:  20
------------------------------------------------------------------------
n:  17  n.digits(prime):  [3, 2]  sd:  2  prime:  7  n mod prime:  3
basic roots:
[1, 8, 12, 16, 24, 25, 31, 32, 36, 40, 48]
root-logs and multiplicities:
[[1, 1], [7, 1], [8, 4], [12, 1], [16, 5], [24, 4], [25, 1], [31, 1], [32, 4], [36, 1], [40, 5], [48, 4]]
total multiplicity of roots:  32
------------------------------------------------------------------------
n:  24  n.digits(prime):  [3, 3]  sd:  2  prime:  7  n mod prime:  3
basic roots:
[1, 8, 12, 16, 24, 25, 31, 32, 36, 40, 48]
root-logs and multiplicities:
[[1, 1], [7, 1], [8, 6], [12, 1], [16,

------------------------------------------------------------------------
n:  171  n.digits(prime):  [3, 3, 3]  sd:  2  prime:  7  n mod prime:  3
basic roots:
[1, 8, 12, 16, 24, 25, 31, 32, 36, 40, 48]
root-logs and multiplicities:
[[1, 1], [7, 1], [8, 48], [12, 1], [16, 49], [24, 48], [25, 1], [31, 1], [32, 48], [36, 1], [40, 49], [48, 48]]
total multiplicity of roots:  296
------------------------------------------------------------------------
n:  178  n.digits(prime):  [3, 4, 3]  sd:  2  prime:  7  n mod prime:  3
basic roots:
[1, 8, 12, 16, 24, 25, 31, 32, 36, 40, 48]
root-logs and multiplicities:
[[1, 1], [7, 1], [8, 50], [12, 1], [16, 51], [24, 50], [25, 1], [31, 1], [32, 50], [36, 1], [40, 51], [48, 50]]
total multiplicity of roots:  308
------------------------------------------------------------------------
n:  185  n.digits(prime):  [3, 5, 3]  sd:  2  prime:  7  n mod prime:  3
basic roots:
[1, 8, 12, 16, 24, 25, 31, 32, 36, 40, 48]
root-logs and multiplicities:
[[1, 1], [7,

In [8]:
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=7
residue=4
sdvalue=6
badn=[]
count=0
flag=1
template=[]
for k in [1..200]:
    n=s[k][0]
    if (n>prime-1)&(mod(n,prime)==residue):
        poly=D(s[k][1])
        polynum=ZR(polynomialnumerator(D(poly)))
        sd=leastSplittingDegree(polynum,prime,bound)
    
        if (sd==sdvalue)&(flag==1):
            template=basicRoots(polynum,prime,sd)
            flag=0
    
        if (sd==sdvalue):
            count=1+count
            br=basicRoots(polynum,prime,sd)
            if br!=template:badn=badn+[n]
            lm=logsAndMultiplicitiesOverField(polynum,prime,sd)
            tm=totalMultiplicityOverField(polynum,prime,sd)
            print "------------------------------------------------------------------------"
            print "n: ",n," n.digits(prime): ",n.digits(prime)," sd: ",sd," prime: ",prime," n mod prime: ",mod(n,prime)
            print "basic roots:";print br
            print "root-logs and multiplicities:";print sorted(lm)
            print "total multiplicity of roots: ",tm
print "&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&"
if count==0:print "null"
print " bad n: ",badn;print "template: ",template

------------------------------------------------------------------------
n:  11  n.digits(prime):  [4, 1]  sd:  6  prime:  7  n mod prime:  4
basic roots:
[19608, 22704, 29412, 39216, 41280, 53664, 58824, 78432, 81528, 88236, 98040, 100104, 112488, 117648]
root-logs and multiplicities:
[[19608, 2], [22704, 1], [29412, 1], [39216, 3], [41280, 1], [53664, 1], [58824, 2], [78432, 2], [81528, 1], [88236, 1], [98040, 3], [100104, 1], [112488, 1], [117648, 2]]
total multiplicity of roots:  22
------------------------------------------------------------------------
n:  18  n.digits(prime):  [4, 2]  sd:  6  prime:  7  n mod prime:  4
basic roots:
[19608, 22704, 29412, 39216, 41280, 53664, 58824, 78432, 81528, 88236, 98040, 100104, 112488, 117648]
root-logs and multiplicities:
[[19608, 4], [22704, 1], [29412, 1], [39216, 5], [41280, 1], [53664, 1], [58824, 4], [78432, 4], [81528, 1], [88236, 1], [98040, 5], [100104, 1], [112488, 1], [117648, 4]]
total multiplicity of roots:  34
----------------

------------------------------------------------------------------------
n:  130  n.digits(prime):  [4, 4, 2]  sd:  6  prime:  7  n mod prime:  4
basic roots:
[19608, 22704, 29412, 39216, 41280, 53664, 58824, 78432, 81528, 88236, 98040, 100104, 112488, 117648]
root-logs and multiplicities:
[[19608, 36], [22704, 1], [29412, 1], [39216, 37], [41280, 1], [53664, 1], [58824, 36], [78432, 36], [81528, 1], [88236, 1], [98040, 37], [100104, 1], [112488, 1], [117648, 36]]
total multiplicity of roots:  226
------------------------------------------------------------------------
n:  137  n.digits(prime):  [4, 5, 2]  sd:  6  prime:  7  n mod prime:  4
basic roots:
[19608, 22704, 29412, 39216, 41280, 53664, 58824, 78432, 81528, 88236, 98040, 100104, 112488, 117648]
root-logs and multiplicities:
[[19608, 38], [22704, 1], [29412, 1], [39216, 39], [41280, 1], [53664, 1], [58824, 38], [78432, 38], [81528, 1], [88236, 1], [98040, 39], [100104, 1], [112488, 1], [117648, 38]]
total multiplicity of roots:

In [9]:
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=7
residue=5
sdvalue=5
badn=[]
count=0
flag=1
template=[]
for k in [1..200]:
    n=s[k][0]
    if (n>prime-1)&(mod(n,prime)==residue):
        poly=D(s[k][1])
        polynum=ZR(polynomialnumerator(D(poly)))
        sd=leastSplittingDegree(polynum,prime,bound)
    
        if (sd==sdvalue)&(flag==1):
            template=basicRoots(polynum,prime,sd)
            flag=0
    
        if (sd==sdvalue):
            count=1+count
            br=basicRoots(polynum,prime,sd)
            if br!=template:badn=badn+[n]
            lm=logsAndMultiplicitiesOverField(polynum,prime,sd)
            tm=totalMultiplicityOverField(polynum,prime,sd)
            print "------------------------------------------------------------------------"
            print "n: ",n," n.digits(prime): ",n.digits(prime)," sd: ",sd," prime: ",prime," n mod prime: ",mod(n,prime)
            print "basic roots:";print br
            print "root-logs and multiplicities:";print sorted(lm)
            print "total multiplicity of roots: ",tm
print "&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&"
if count==0:print "null"
print " bad n: ",badn;print "template: ",template

------------------------------------------------------------------------
n:  12  n.digits(prime):  [5, 1]  sd:  5  prime:  7  n mod prime:  5
basic roots:
[1513, 2188, 2617, 2801, 5602, 6376, 6913, 8403, 9916, 11020, 11204, 14005, 14779, 16806]
root-logs and multiplicities:
[[1513, 1], [2188, 1], [2617, 1], [2801, 2], [5602, 3], [6376, 1], [6913, 1], [8403, 2], [9916, 1], [10591, 1], [11020, 1], [11204, 2], [14005, 3], [14779, 1], [15316, 1], [16806, 2]]
total multiplicity of roots:  24
------------------------------------------------------------------------
n:  19  n.digits(prime):  [5, 2]  sd:  5  prime:  7  n mod prime:  5
basic roots:
[1513, 2188, 2617, 2801, 5602, 6376, 6913, 8403, 9916, 11020, 11204, 14005, 14779, 16806]
root-logs and multiplicities:
[[1513, 1], [2188, 1], [2617, 1], [2801, 4], [5602, 5], [6376, 1], [6913, 1], [8403, 4], [9916, 1], [10591, 1], [11020, 1], [11204, 4], [14005, 5], [14779, 1], [15316, 1], [16806, 4]]
total multiplicity of roots:  36
----------------

------------------------------------------------------------------------
n:  131  n.digits(prime):  [5, 4, 2]  sd:  5  prime:  7  n mod prime:  5
basic roots:
[1513, 2188, 2617, 2801, 5602, 6376, 6913, 8403, 9916, 11020, 11204, 14005, 14779, 16806]
root-logs and multiplicities:
[[1513, 1], [2188, 1], [2617, 1], [2801, 36], [5602, 37], [6376, 1], [6913, 1], [8403, 36], [9916, 1], [10591, 1], [11020, 1], [11204, 36], [14005, 37], [14779, 1], [15316, 1], [16806, 36]]
total multiplicity of roots:  228
------------------------------------------------------------------------
n:  138  n.digits(prime):  [5, 5, 2]  sd:  5  prime:  7  n mod prime:  5
basic roots:
[1513, 2188, 2617, 2801, 5602, 6376, 6913, 8403, 9916, 11020, 11204, 14005, 14779, 16806]
root-logs and multiplicities:
[[1513, 1], [2188, 1], [2617, 1], [2801, 38], [5602, 39], [6376, 1], [6913, 1], [8403, 38], [9916, 1], [10591, 1], [11020, 1], [11204, 38], [14005, 39], [14779, 1], [15316, 1], [16806, 38]]
total multiplicity of roots:

In [10]:
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=7
residue=6
sdvalue=1
badn=[]
count=0
flag=1
template=[]
for k in [1..200]:
    n=s[k][0]
    if (n>prime-1)&(mod(n,prime)==residue):
        poly=D(s[k][1])
        polynum=ZR(polynomialnumerator(D(poly)))
        sd=leastSplittingDegree(polynum,prime,bound)
    
        if (sd==sdvalue)&(flag==1):
            template=basicRoots(polynum,prime,sd)
            flag=0
    
        if (sd==sdvalue):
            count=1+count
            br=basicRoots(polynum,prime,sd)
            if br!=template:badn=badn+[n]
            lm=logsAndMultiplicitiesOverField(polynum,prime,sd)
            tm=totalMultiplicityOverField(polynum,prime,sd)
            print "------------------------------------------------------------------------"
            print "n: ",n," n.digits(prime): ",n.digits(prime)," sd: ",sd," prime: ",prime," n mod prime: ",mod(n,prime)
            print "basic roots:";print br
            print "root-logs and multiplicities:";print sorted(lm)
            print "total multiplicity of roots: ",tm
print "&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&"
if count==0:print "null"
print " bad n: ",badn;print "template: ",template

------------------------------------------------------------------------
n:  13  n.digits(prime):  [6, 1]  sd:  1  prime:  7  n mod prime:  6
basic roots:
[1, 2, 3, 4, 5, 6]
root-logs and multiplicities:
[[1, 4], [2, 4], [3, 4], [4, 4], [5, 4], [6, 4]]
total multiplicity of roots:  24
------------------------------------------------------------------------
n:  20  n.digits(prime):  [6, 2]  sd:  1  prime:  7  n mod prime:  6
basic roots:
[1, 2, 3, 4, 5, 6]
root-logs and multiplicities:
[[1, 6], [2, 6], [3, 6], [4, 6], [5, 6], [6, 6]]
total multiplicity of roots:  36
------------------------------------------------------------------------
n:  27  n.digits(prime):  [6, 3]  sd:  1  prime:  7  n mod prime:  6
basic roots:
[1, 2, 3, 4, 5, 6]
root-logs and multiplicities:
[[1, 8], [2, 8], [3, 8], [4, 8], [5, 8], [6, 8]]
total multiplicity of roots:  48
------------------------------------------------------------------------
n:  34  n.digits(prime):  [6, 4]  sd:  1  prime:  7  n mod prime:  6
